# 0) Web Scraping - US Hit Streaming Songs Project

__The following Notebook is the precursor to a four-part series. This Notebook, specifically, collects data from HITS Daily Double's [Streaming Songs Chart](https://hitsdailydouble.com/streaming_songs) and adds it to [this](https://docs.google.com/spreadsheets/d/165OdLYjLt4AgeqP5S5PunRonDkpp28nueHLFv994bPk/edit#gid=0) Google Sheet__. The steps taken are the following:
- Webscraping the Data
- Importing the Google Sheet and turning it into a dataframe
- Merging data
- Editing the Google Sheet dataframe
- Checking the newly edited Google Sheet dataframe
- Saving final dataframe to Google Sheet

__Overview of the process__: Once I completed parts 1-4 of this project, I decided to go back and improve how I collected data every week by __webscraping weekly streaming data instead of manually inputing it onto Google Sheets (as I did for these last 5 years before I knew how to code)__. In order to do so, this project utilizes [Beautiful Soup](https://beautiful-soup-4.readthedocs.io/en/latest/), a Python library that pulls data out of HTML and XML files. Once Beautiful Soup is imported, we create two dataframes, one taken straight from HITS Daily Double's chart and the other one being a cleaned-up version of the first dataframe. Then, we import our [Google Sheet](https://docs.google.com/spreadsheets/d/165OdLYjLt4AgeqP5S5PunRonDkpp28nueHLFv994bPk/edit#gid=0) and join it with the newly-cleaned-up dataframe. Finally, we save the resulting dataframe to the Google Sheet. 

__In the far future, I aim to utilize Wayback Machine to automate the entire data collection for every weekly chart since 2017, so that my resulting data can look exactly like the Google Sheet I currently have__. The reason for this taking a while is because so many data entries in my Google Sheet are __edited__ versions of the numbers that come from HITS Daily Double. Sometimes HITS Daily Double has notable mistakes such as accidentally omitting a song from the charts one week but having it present the following week. Luckily, HITS Daily Double also shows weekly changes in streams by percentage points to help correct these errors.

# Webscraping the Data

In [1]:
# Import libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd

#to ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
#get url and create our "soup"
url = "https://hitsdailydouble.com/streaming_songs"
page = requests.get(url)
soup = BeautifulSoup(page.text, 'lxml')

In [3]:
#to get chart week and convert it into a string
from datetime import datetime
from datetime import timedelta

week_of = str(soup.findAll('table')[2].find_all('span')[0]).split('> ')[1].split(' (WEEK')[0]

week_of = str(datetime.strptime(week_of, '%m/%d/%Y').date() - timedelta(days=7))

print(week_of)

2022-11-18


In [4]:
# Collecting data and creating a temporary df
df_temp = pd.DataFrame(columns=['col', 'strong', 'span'])

#get information from the website
for row in soup.findAll('table')[3].findAll('tr'): 
    column = row.find_all('td')
    all_strong=row.find('strong')
    all_span=row.find_all('span')
            
    df_temp = df_temp.append({'col':column, 'strong':all_strong, 
                'span':all_span},ignore_index=True)
df_temp

,col,strong,span
0,"[[LW], [TW], [Artist | Title | Label], [Stream...",None,[]
1,"[[\n1 ], [1 ], [[<tr><td><img class=""hits_albu...","[27,195,130]","[[DRAKE & 21 SAVAGE | RICH FLEX], [OVO/REPUBLI..."
2,"[[[]], [[DRAKE & 21 SAVAGE | RICH FLEX], [], [...",None,"[[DRAKE & 21 SAVAGE | RICH FLEX], [OVO/REPUBLI..."
3,"[[\n2 ], [2 ], [[<tr><td><img class=""hits_albu...","[21,404,162]","[[TAYLOR SWIFT | ANTI-HERO], [REPUBLIC (UMG)]]"
4,"[[[]], [[TAYLOR SWIFT | ANTI-HERO], [], [REPUB...",None,"[[TAYLOR SWIFT | ANTI-HERO], [REPUBLIC (UMG)]]"
...,...,...,...
96,"[[\n--], [48 ], [[LUKE COMBS | THE KIND OF LOV...",None,"[[LUKE COMBS | THE KIND OF LOVE WE MAKE], [RIV..."
97,"[[[<table class=""hits_album_chart_full""><tr><t...","[7,909,674]",[[PERRY COMO | IT'S BEGINNING TO LOOK A LOT LI...
98,"[[\n--], [49 ], [[PERRY COMO | IT'S BEGINNING ...",None,[[PERRY COMO | IT'S BEGINNING TO LOOK A LOT LI...
99,"[[[<table class=""hits_album_chart_full""><tr><t...","[7,774,188]","[[ONEREPUBLIC | I AIN'T WORRIED], [INTERSCOPE ..."


This table looks odd (due to the fact that the the format for the title row, songs of rank 1-5, and songs of rank 6-50 have different formats). In order to extract the data we want, run the following:

In [5]:
#creating a list of indeces that have the following variables: last week rank, current week rank, artist, song title, record labels
list_of_indeces_for_ranking_nums = []

#the indeces with the variables we want are on odd numbers for the first 5 ranked songs
for i in range(1,11,2):
    list_of_indeces_for_ranking_nums.append(i)

#the indeces with the variables we want are on even numbers all other ranks
for i in range(12,101,2):
    list_of_indeces_for_ranking_nums.append(i)

#to confirm that there are 50 indeces
len(list_of_indeces_for_ranking_nums)

50

In [6]:
lw = [] #last week's ranks
tw = [] #this week's ranks
artist_list = []
song_title_list = []
label = []

for i in list_of_indeces_for_ranking_nums:
    #for weekly ranks
    lw.append(df_temp.iloc[i,0][0].text.strip('\n'))
    tw.append(df_temp.iloc[i,0][1].text.strip('\n'))
    
    #for artist name
    artist = df_temp.iloc[i,2][0].text.split(" | ")[0]
    artist = artist.split(" &")[0] #gets rid of everything after " &"
    artist = artist.split(" FEAT.")[0] #gets rid of everthing after" FEAT."
    artist_list.append(artist)
    
    #for record label
    label.append(df_temp.iloc[i,2][1].text)
    
    #for song title
    song_title_list.append(df_temp.iloc[i,2][0].text.split(" | ")[1])

In [7]:
#finding information for the weekly streams column
list_of_stream_nums = []

#weekly streams are found on every odd index (after 1) 
for i in range(1,101,2):
    list_of_stream_nums.append(df_temp.iloc[i][1].text)

In [8]:
#finding information for the percent change column
change_pcnt = []

#depending on whether song is in the top 5 or not, % change is found in different locations of df_temp
for i in range(1,11,2):
    change_pcnt.append(df_temp.iloc[i][0][6].text)

for i in range(11,101,2):
    change_pcnt.append(df_temp.iloc[i][0][5].text)

In [9]:
#create final dataframe from HITS Daily Double
df =  pd.DataFrame(list(zip(lw,tw,artist_list,song_title_list,label,
            list_of_stream_nums,change_pcnt)),
        columns=['lw', 'tw', 'artist','title','label','streams'
                ,'change'])

#change streams from strings to integers
df['streams'] = df['streams'].str.replace(',','').astype('int')

#get rid of apostraphes and "...?" from artist names and song titles
df['artist'] = df['artist'].str.replace("’","").str.replace("'","").str.lower()
df['title'] = df['title'].str.replace("’","").str.replace("'","").replace("'","").str.lower()
df['title'] = df['title'].str.strip("…?")

#get rid of everything in parentheses as well as whitespaces
df['title'] = df['title'].str.replace(r"\(.*\)","")
df['title'] = df['title'].str.strip(" ")

#to make sure we have 50 songs
print(len(df))

df

50


,lw,tw,artist,title,label,streams,change
0,1,1,drake,rich flex,OVO/REPUBLIC (UMG),27195130,-20%
1,2,2,taylor swift,anti-hero,REPUBLIC (UMG),21404162,-12%
2,5,3,sam smith,unholy,CAPITOL (UMG),16049494,-6%
3,7,4,steve lacy,bad habit,L-M/RCA (SME),15327473,-3%
4,19,5,mariah carey,all i want for christmas is you,COLUMBIA (SME),14507461,27%
5,17,6,lil uzi vert,just wanna rock,GENERATION NOW/ATLANTIC (WMG),14448260,23%
6,33,7,brenda lee,rockin around the christmas tree,MCA NASHVILLE (UMG),13054017,37%
7,12,8,zach bryan,something in the orange,WARNER (WMG),13045498,-3%
8,3,9,drake,major distribution,OVO/REPUBLIC (UMG),12768328,-33%
9,37,10,bobby helms,jingle bell rock,GEFFEN (UMG),12494883,38%


# Importing the Google Sheet

In [10]:
#to access google sheets with python
import gspread
from google.oauth2 import service_account
from oauth2client.service_account import ServiceAccountCredentials

#service account
sa = gspread.service_account(filename="gspread_service_account.json")

#sheet
sh = sa.open("HITS Streaming Songs")

#worksheet
wks = sh.worksheet("HITS Streaming Songs")

#uses all values in the worksheet for the data frame
gsheets_df = pd.DataFrame(wks.get_all_values())

header = gsheets_df.iloc[0] #isolate first row as header
gsheets_df = gsheets_df[1:] #get rid of header in original df
gsheets_df.columns = header

#set 'index' as index
gsheets_df = gsheets_df.set_index('index')

gsheets_df['title'] = gsheets_df['title'].str.rstrip(' ')
gsheets_df['artist'] = gsheets_df['artist'].str.rstrip(' ')
gsheets_df['album'] = gsheets_df['album'].str.rstrip(' ')

#convert index from string to int
gsheets_df.index = gsheets_df.index.astype(int)

#fill in blank values with "0" (can't convert NaN to integer)
gsheets_df.iloc[:,4:] = gsheets_df.iloc[:,4:].replace(r'^\s*$', "0", regex=True)

#convert all numbers-as-strings to numbers from row 4 in python's index and onwards
for col_name in gsheets_df.columns.to_list()[4:len(gsheets_df.columns.to_list())-1]:
    #get rid of strings' commas and convert strings to integers
    gsheets_df[col_name] = gsheets_df[col_name].str.replace(',','').astype('int')
    
#turn peak_date strings to date format
gsheets_df['peak_date'] = pd.to_datetime(gsheets_df['peak_date'])

#turn zeros from string to int
gsheets_df.iloc[:,4:] = gsheets_df.iloc[:,4:].replace('0', 0, regex=True)

gsheets_df


,title,artist,album,peak_date,streams_2017_to_present,total_streams_2020s,peak_week_streams,current_year,2022-11-11,2022-11-04,...,total_2022,total_2021,total_2020,total_2019,total_2018,total_2017,units,holiday_music,remix_version,playlist_index
index,,,,,,,,,,,,,,,,,,,,,
1,Closer,The Chainsmokers,Closer,2016-09-09,10948414,0,5528407,0,0,0,...,0,0,0,0,0,10948414,36856,0,0,46
2,Fake Love,Drake,More Life,2016-11-04,40163498,0,7800314,0,0,0,...,0,0,0,0,0,40163498,52002,0,0,0
3,Starboy,The Weeknd,Starboy,2016-11-25,10554623,0,5225061,0,0,0,...,0,0,0,0,0,10554623,34834,0,0,48
4,Reminder,The Weeknd,Starboy,2016-11-25,5229932,0,5229932,0,0,0,...,0,0,0,0,0,5229932,34866,0,0,49
5,Both (feat. Drake),Gucci Mane,The Return of East Atlanta Santa,2016-12-16,23375700,0,6072680,0,0,0,...,0,0,0,0,0,23375700,40485,0,0,51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2313,Jumbotron Shit Poppin,Drake,Her Loss,2022-11-04,34637833,34637833,24188702,34637833,10449131,24188702,...,34637833,0,0,0,0,0,161258,0,0,0
2314,More M's,Drake,Her Loss,2022-11-04,29365063,29365063,21353198,29365063,8011865,21353198,...,29365063,0,0,0,0,0,142355,0,0,0
2315,3AM on Glenwood,21 Savage,Her Loss,2022-11-04,16301124,16301124,16301124,16301124,0,16301124,...,16301124,0,0,0,0,0,108674,0,0,0


From here, we create a copy of the Google Sheet dataframe and further edit that copied dataframe.

In [11]:
#make a copy of the Google Sheet
copy_df = gsheets_df.copy()

#get rid of everything in parenthesis
copy_df['title'] = copy_df['title'].str.replace(r"\(.*\)","")

#to get the first item of a list
def get_first_list_item(df):
    for i in range(1,len(df)+1):
        #print(df['title'][i][0])
        df['title'][i] = df['title'][i][0]

#split for dashes (with a space before it or else songs like '1-800-273-8255' will be affected)
copy_df['title'] = copy_df['title'].str.split(" -")
get_first_list_item(copy_df)

#getting rid of any whitespace at the end of a song title
copy_df['title'] = copy_df['title'].str.strip(" ")

#for merging purposes (merging Google Sheets with HITS Daily Double's charts)
copy_df['title'][624] = 'Have A Holly Jolly Christmas'
copy_df['title'] = copy_df['title'].str.strip("...?")

copy_df['artist'] = copy_df['artist'].str.replace("’","").str.replace("'","").str.lower()
copy_df['title'] = copy_df['title'].str.replace("’","").str.replace("'","").str.lower()

#to remove_accents
from unidecode import unidecode
copy_df['artist'] = copy_df['artist'].apply(unidecode)
copy_df['title'] = copy_df['title'].apply(unidecode)

copy_df

,title,artist,album,peak_date,streams_2017_to_present,total_streams_2020s,peak_week_streams,current_year,2022-11-11,2022-11-04,...,total_2022,total_2021,total_2020,total_2019,total_2018,total_2017,units,holiday_music,remix_version,playlist_index
index,,,,,,,,,,,,,,,,,,,,,
1,closer,the chainsmokers,Closer,2016-09-09,10948414,0,5528407,0,0,0,...,0,0,0,0,0,10948414,36856,0,0,46
2,fake love,drake,More Life,2016-11-04,40163498,0,7800314,0,0,0,...,0,0,0,0,0,40163498,52002,0,0,0
3,starboy,the weeknd,Starboy,2016-11-25,10554623,0,5225061,0,0,0,...,0,0,0,0,0,10554623,34834,0,0,48
4,reminder,the weeknd,Starboy,2016-11-25,5229932,0,5229932,0,0,0,...,0,0,0,0,0,5229932,34866,0,0,49
5,both,gucci mane,The Return of East Atlanta Santa,2016-12-16,23375700,0,6072680,0,0,0,...,0,0,0,0,0,23375700,40485,0,0,51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2313,jumbotron shit poppin,drake,Her Loss,2022-11-04,34637833,34637833,24188702,34637833,10449131,24188702,...,34637833,0,0,0,0,0,161258,0,0,0
2314,more ms,drake,Her Loss,2022-11-04,29365063,29365063,21353198,29365063,8011865,21353198,...,29365063,0,0,0,0,0,142355,0,0,0
2315,3am on glenwood,21 savage,Her Loss,2022-11-04,16301124,16301124,16301124,16301124,0,16301124,...,16301124,0,0,0,0,0,108674,0,0,0


# Merge the data
The following merges the data from the dataframe from HITS Daily Double and the copied Google Sheet dataframe and turns it into a dictionary.

In [12]:
#merge the data
merged_df = pd.merge(copy_df, df, how='left', on=["title","artist"])

#print how many streams are above 0 (should be 50)
print(len(merged_df[merged_df['streams']>0].sort_values('streams',ascending=False)))

#show sorted dataframe with all streams above 0
merged_df[merged_df['streams']>0].sort_values('streams',ascending=False)



50


,title,artist,album,peak_date,streams_2017_to_present,total_streams_2020s,peak_week_streams,current_year,2022-11-11,2022-11-04,...,total_2017,units,holiday_music,remix_version,playlist_index,lw,tw,label,streams,change
2300,rich flex,drake,Her Loss,2022-11-04,91929923,91929923,58141838,91929923,33788085,58141838,...,0,387612,0,0,"2,317",1,1,OVO/REPUBLIC (UMG),27195130.0,-20%
2271,anti-hero,taylor swift,Midnights (3am Edition),2022-10-21,133771966,133771966,52318794,133771966,23367720,26182959,...,0,348792,0,0,"2,286",2,2,REPUBLIC (UMG),21404162.0,-12%
2241,unholy,sam smith,Unholy (feat. Kim Petras),2022-09-23,159943915,159943915,22558529,159943915,17125577,18047475,...,0,150390,0,0,"2,237",5,3,CAPITOL (UMG),16049494.0,-6%
2165,bad habit,steve lacy,Gemini Rights,2022-08-12,334034261,334034261,19494196,334034261,16792328,15893030,...,0,129961,0,0,0,7,4,L-M/RCA (SME),15327473.0,-3%
210,all i want for christmas is you,mariah carey,Merry Christmas,2020-12-11,351971094,219615410,32605807,10561894,10561894,0,...,19664559,217372,1,0,23,19,5,COLUMBIA (SME),14507461.0,27%
2289,just wanna rock,lil uzi vert,Just Wanna Rock,2022-11-04,50466967,50466967,14760571,50466967,11746424,14760571,...,0,98404,0,0,0,17,6,GENERATION NOW/ATLANTIC (WMG),14448260.0,23%
213,rockin around the christmas tree,brenda lee,Merry Christmas From Brenda Lee,2020-12-11,262949554,181258533,29495103,9173780,9173780,0,...,11632783,196634,1,0,22,33,7,MCA NASHVILLE (UMG),13054017.0,37%
2030,something in the orange,zach bryan,Something in the Orange,2022-10-14,324322135,324322135,15782575,324322135,14644399,15013019,...,0,105217,0,0,"2,242",12,8,WARNER (WMG),13045498.0,-3%
2301,major distribution,drake,Her Loss,2022-11-04,64280144,64280144,45494391,64280144,18785753,45494391,...,0,303296,0,0,"2,316",3,9,OVO/REPUBLIC (UMG),12768328.0,-33%
622,jingle bell rock,bobby helms,Jingle Bell Rock/Captain Santa Claus (And His ...,2020-12-11,245378451,170705850,26725703,8762443,8762443,0,...,0,178171,1,0,21,37,10,GEFFEN (UMG),12494883.0,38%


In [13]:
#convert merged_df for dictionary
merged_dct = merged_df[merged_df['streams']>0].sort_values('streams',ascending=False).iloc[:,-2::2].to_dict()

# Edit the Google Sheet Dataframe

In [14]:
#create a new column of the latest week and set all values to 0
gsheets_df[week_of] = 0

In [15]:
#change streams for new week if index is in keys of the dict made from merged_df
for i in range(0,len(gsheets_df)):
    if gsheets_df.index[i-1] in list(merged_dct['streams'].keys()):
        gsheets_df[week_of][i+1] = merged_dct['streams'][i]

#rearrange the columns so that new week is after current_year column
cols = gsheets_df.columns.tolist()
cols = cols[:8] + [str(week_of)] + cols[8:-1]
gsheets_df = gsheets_df[cols]

gsheets_df.head()

,title,artist,album,peak_date,streams_2017_to_present,total_streams_2020s,peak_week_streams,current_year,2022-11-18,2022-11-11,...,total_2022,total_2021,total_2020,total_2019,total_2018,total_2017,units,holiday_music,remix_version,playlist_index
index,,,,,,,,,,,,,,,,,,,,,
1,Closer,The Chainsmokers,Closer,2016-09-09,10948414,0,5528407,0,0,0,...,0,0,0,0,0,10948414,36856,0,0,46
2,Fake Love,Drake,More Life,2016-11-04,40163498,0,7800314,0,0,0,...,0,0,0,0,0,40163498,52002,0,0,0
3,Starboy,The Weeknd,Starboy,2016-11-25,10554623,0,5225061,0,0,0,...,0,0,0,0,0,10554623,34834,0,0,48
4,Reminder,The Weeknd,Starboy,2016-11-25,5229932,0,5229932,0,0,0,...,0,0,0,0,0,5229932,34866,0,0,49
5,Both (feat. Drake),Gucci Mane,The Return of East Atlanta Santa,2016-12-16,23375700,0,6072680,0,0,0,...,0,0,0,0,0,23375700,40485,0,0,51


In [16]:
for i in range(len(gsheets_df)):
    #updating sums
    gsheets_df['streams_2017_to_present'][i+1] = sum(gsheets_df.iloc[i,8:-10])
    gsheets_df['total_streams_2020s'][i+1] = sum(gsheets_df.iloc[i,8:-152])
    gsheets_df['total_2022'][i+1] = sum(gsheets_df.iloc[i,8:-257])   
    gsheets_df['current_year'][i+1] = gsheets_df['total_2022'][i+1] 
    
    #updating peak weekly streams
    gsheets_df['peak_week_streams'][i+1] = max(gsheets_df.iloc[i,8:-10])
    gsheets_df['units'][i+1] = (gsheets_df['peak_week_streams'][i+1])/150
    
    #change peak date if this week is the max
    if gsheets_df[week_of][i+1] == gsheets_df['peak_week_streams'][i+1]:
        print(gsheets_df['title'][i+1], "by", gsheets_df['artist'][i+1], "reached a new peak this week!")
        gsheets_df['peak_date'][i+1] = week_of

golden hour by JVKE reached a new peak this week!
Made You Look by Meghan Trainor reached a new peak this week!


# Final check of the newly updated Dataframe

In [17]:
gsheets_df.sort_values(week_of, ascending=False).head(50)

,title,artist,album,peak_date,streams_2017_to_present,total_streams_2020s,peak_week_streams,current_year,2022-11-18,2022-11-11,...,total_2022,total_2021,total_2020,total_2019,total_2018,total_2017,units,holiday_music,remix_version,playlist_index
index,,,,,,,,,,,,,,,,,,,,,
2301,Rich Flex,Drake,Her Loss,2022-11-04,119125053,119125053,58141838,119125053,27195130,33788085,...,119125053,0,0,0,0,0,387612,0,0,"2,317"
2272,Anti-Hero,Taylor Swift,Midnights (3am Edition),2022-10-21,155176128,155176128,52318794,155176128,21404162,23367720,...,155176128,0,0,0,0,0,348791,0,0,"2,286"
2242,Unholy (feat. Kim Petras),Sam Smith,Unholy (feat. Kim Petras),2022-09-23,175993409,175993409,22558529,175993409,16049494,17125577,...,175993409,0,0,0,0,0,150390,0,0,"2,237"
2166,Bad Habit,Steve Lacy,Gemini Rights,2022-08-12,349361734,349361734,19494196,349361734,15327473,16792328,...,349361734,0,0,0,0,0,129961,0,0,0
211,All I Want for Christmas Is You,Mariah Carey,Merry Christmas,2020-12-11,366478556,234122871,32605807,25069355,14507461,10561894,...,25069355,104834801,104218715,50040694,62650432,19664559,217372,1,0,23
2290,Just Wanna Rock,Lil Uzi Vert,Just Wanna Rock,2022-11-04,64915227,64915227,14760571,64915227,14448260,11746424,...,64915227,0,0,0,0,0,98403,0,0,0
214,Rockin' Around The Christmas Tree,Brenda Lee,Merry Christmas From Brenda Lee,2020-12-11,276003571,194312550,29495103,22227797,13054017,9173780,...,22227797,85138635,86946118,44906111,25152127,11632783,196634,1,0,22
2031,Something in the Orange,Zach Bryan,Something in the Orange,2022-10-14,337367633,337367633,15782575,337367633,13045498,14644399,...,337367633,0,0,0,0,0,105217,0,0,"2,242"
2302,Major Distribution,Drake,Her Loss,2022-11-04,77048472,77048472,45494391,77048472,12768328,18785753,...,77048472,0,0,0,0,0,303295,0,0,"2,316"


# Save to Google Sheet

In [18]:
#to edit google sheets from python
from df2gspread import df2gspread as d2g

#reset index and convert all integers/float/dates to strings in preparation to convert to a Google Shet
final_df = gsheets_df.reset_index()

url = "https://docs.google.com/spreadsheets/d/165OdLYjLt4AgeqP5S5PunRonDkpp28nueHLFv994bPk/edit#gid=0"

spreadsheet_key = url.split("/")[-2]

wks_name = 'Automated Sheet - Pre-spotipy'

scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']

credentials = ServiceAccountCredentials.from_json_keyfile_name('gspread_service_account.json', scope)

gc = gspread.authorize(credentials)

d2g.upload(final_df, spreadsheet_key, wks_name, credentials = credentials, row_names=False)

<Worksheet 'Automated Sheet - Pre-spotipy' id:1134073350>

From here I take the new information and copy+paste it into the "HITS Streaming Songs" sheet. This saves me a lot of time compared to before! This process has to be done every week, but luckily the owner of this project is extremely dedicated to staying on top of it! :)
## Now we can move on to part 1 of this project to merge all our collected data with Spotify's API data :)